# Version

In [ ]:
"""
Author: Zhang Lu
Version: 1.0.0
Date:2025-02-14
Description: Happy Valentine's Day.
"""

# Packages & Parameters

In [ ]:
import os
import importlib
import pickle

In [ ]:
import Complex_Utility
importlib.reload(Complex_Utility)
from Complex_Utility import *

import config
importlib.reload(config)
from config import *

In [ ]:
os.chdir(f)
os.getcwd()

# Load

## Load data

In [ ]:
df = pd.read_pickle(score_file)

## Load model

In [ ]:
model = pickle.load(file = open(pkl_file,'rb'))
model_var = model.feature_name_
len(model_var)

## Data preparation

In [ ]:
'''离散变量处理'''
for col in list_dis:
    df[col] = df[col].astype('category')

In [ ]:
# data subgroups
split_dfs = df_split(df, samp_col)
train_x_dist = split_dfs['01.train']
train_y = split_dfs['01.train'][y]
test_x_dist = split_dfs['02.test']
test_y = split_dfs['02.test'][y]
vldt_x_dist = split_dfs['03.vldt']
vldt_y = split_dfs['03.vldt'][y]

# Choose top vars to describe statistics

In [ ]:
'''choose top few (10) Vars to describe'''
df_imp = pd.DataFrame()
index = 0
for i, j in zip(model.feature_name_, model.feature_importances_):
    # print(i, j)
    df_imp.loc[index, 'col_name'] = i
    df_imp.loc[index, 'imp_val'] = j
    index = index + 1
df_imp = df_imp.sort_values(by=['imp_val'], ascending=False)
df_imp_top = list(df_imp.head(10).col_name)

print(df_imp_top.shape)

# Bin top vars to be described

In [ ]:
train_slc1_update_woe, slc1_bins_update, slc1_c_update, slc1_t_update, train_slc1_woe, slc1_bins, slc1_c, slc1_t = discrete_type(
    train_x_dist, train_y, df_imp_top, bin_set=False)

df_slc2_bin = slc1_c_update.transform(df[df_imp_top],labels=True)
df_slc2_woe_xy = pd.concat([df_slc2_bin, df[y,samp_col]], axis=1)
'''show df as Bin name'''
train_slc2_bin = slc1_c_update.transform(train_x_dist[df_imp_top],labels=True)
test_slc2_bin = slc1_c_update.transform(test_x_dist[df_imp_top],labels=True)
vldt_slc2_bin = slc1_c_update.transform(vldt_x_dist[df_imp_top],labels=True)
'''show df as WOE'''
train_slc2_woe_tot = slc1_t_update.transform(train_slc2_bin)
test_slc2_woe_tot = slc1_t_update.transform(test_slc2_bin)
vldt_slc2_woe_tot = slc1_t_update.transform(vldt_slc2_bin)

# Write the report

In [ ]:
import Report_Utility
importlib.reload(Report_Utility)
from Report_Utility import *

In [ ]:
file_nm=part_nm
tgt_col = y

In [ ]:
#入模变量-变量相关性
model_var_corr(wb, train_x_dist[df_imp_top], '%s_入模变量_变量相关性' % file_nm)
#样本分布情况
sample_distribute(wb,
                  ds_all,
                  '%s_样本分布情况' % file_nm,
                  tgt_col,
                  samp_type=samp_col,
                  yearmonth=yearmonth)
#入模变量概况
model_var_stat = model_vars_analyse(wb, tgt_col,train_x_dist[df_imp_top], test_x_dist[df_imp_top],
                                        vldt_x_dist[df_imp_top], train_y, test_y,
                                        vldt_y,
                                        'output/%s_入模变量概况.csv' % file_nm,
                                        '%s_入模变量概况' % file_nm)
#入模变量-单变量分析
woe_update = slc1_t_update.export()
model_vars_sheet2(wb, woe_update, df_imp_top, train_slc1_woe, test_slc2_woe_tot,
                  vldt_slc2_woe_tot, train_y, test_y, vldt_y,
                  'output/%s_双变量分析结果.csv' % file_nm, '%s_双变量分析结果' % file_nm)
#入模变量-单变量分析图
bin_plt_export2(wb, train_x_dist[df_imp_top], df_slc2_woe_xy,
                '%s_入模变量_单变量分析图' % file_nm, model_var_stat, samp_col, tgt_col)
#模型结果-剩余报告
model_result_all(wb, model, file_nm, yearmonth, tgt_col,
                 train_x_dist, train_y,
                 test_x_dist, test_y,
                 vldt_x_dist, vldt_y)

In [ ]:
wb.close()
del wb